The "update_lamda" function is problamatic. If the first step in the iterations has a lower chi-square, then lambda =0 will be given to the function and lamda = 0 will come out. If this happens in the next steps, thn lambda will not change at all. 

Why are the chi-square values very low?

Should I feed different m0 to these three?

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def gauss (m, x):
    #get y=a+b*exp((x-x0)^2/2sig^2)
    #this needs to be changed, I do not need the first two params!
    #a = m[0]
    #b = m[1]
    x0 = m[0]
    sig = m[1]
    expvec = np.exp(-0.5*(x-x0)**2/sig**2)
    
    #y = a+b*expvec
    y = expvec
    derivs = np.empty([len(x), len(m)])
    
    #derivs[:,0] = 1
    #derivs[:,1] = expvec
    
    #derivs[:, 0] = b*(x-x0)*2/(2*sig**2)*expvec
    derivs[:, 0] = (x-x0)*2/(2*sig**2)*expvec
    #derivs[:, 1] = b*(0.5*(x-x0)**2)*2/sig**3*expvec
    derivs[:, 1] = (0.5*(x-x0)**2)*2/sig**3*expvec

    return y,derivs

def update_lamda(lamda, success):
    if success:
        lamda=lamda/1.5
        if lamda<0.5:
            lamda=0
    else:
        if lamda==0:
            lamda=1
        else:
            lamda=lamda*1.5**2
    return lamda

def get_matrices(m, fun, x, y, Ninv):
    model,derivs=fun(m,x)
    r=y-model
    if Ninv is None:
        lhs=derivs.T@derivs
        rhs=derivs.T@r
        chisq=np.sum(r**2)
    else:
        lhs=derivs.T@Ninv@derivs
        rhs=derivs.T@(Ninv@r)
        chisq=r.T@Ninv@r
    return chisq,lhs,rhs

def linv(mat,lamda):
    mat=mat+lamda*np.diag(np.diag(mat))
    return np.linalg.inv(mat)

def fit_lm(m, fun, x, y, Ninv, niter=10, chitol=0.001):
#levenberg-marquardt fitter that supports noise
    lamda=0
    chisq,lhs,rhs=get_matrices(m,fun,x,y,Ninv)
    for i in range(niter):
        lhs_inv=linv(lhs,lamda)
        dm=lhs_inv@rhs
        chisq_new,lhs_new,rhs_new=get_matrices(m+dm,fun,x,y,Ninv)
        if chisq_new<chisq:  
            #accept the step
            #check if we think we are converged - for this, check if 
            #lamda is zero (i.e. the steps are sensible), and the change in 
            #chi^2 is very small - that means we must be very close to the
            #current minimum
            if lamda==0:
                if (np.abs(chisq-chisq_new)<chitol):
                    #print(np.abs(chisq-chisq_new))
                    print('Converged after ',i,' iterations of LM')
                    return m+dm
            chisq=chisq_new
            lhs=lhs_new
            rhs=rhs_new
            m = m+dm
            lamda=update_lamda(lamda,True)
            
        else:
            lamda=update_lamda(lamda,False)
        print('on iteration ', i,' chisq is ', chisq,' with step ', dm,' and lamda ', lamda)
    return m

def chi_square(obs, exp): # to measure the quality of fit (cacualte the bias)
    return np.sum(((obs-exp)**2)/exp)

In [3]:
#defining the noise matrix
def noise_matrix(n, a, sigma):
    N = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if i == j:
                N[i, j] = a * np.exp((-1 * (i - j)**2)/(2 * sigma**2)) + (1 - a)
            else:
                N[i, j] = a * np.exp((-1 * (i - j)**2)/(2 * sigma**2))
    return N

# a

In [4]:
#generating the data
x = np.arange(0, 1000, 1)
m_true = np.asarray([500, 50]) #the original set of params to generate data
m0 = m_true + np.random.randn(len(m_true))*10
y, derivs = gauss(m_true, x) #the noise-free data

In [5]:
a = [0.1, 0.5, 0.9]
sigma = [0.1, 0.5, 0.9]

N_1 = noise_matrix (len(x), a[0], sigma[0])
N_2 = noise_matrix (len(x), a[1], sigma[1])
N_3 = noise_matrix (len(x), a[2], sigma[2])

N_1_inv = np.linalg.inv(N_1)
N_2_inv = np.linalg.inv(N_2)
N_3_inv = np.linalg.inv(N_3)

In [6]:
m_fit_1 = fit_lm(m0, gauss, x, y, N_1, niter=10)
m_fit_2 = fit_lm(m0, gauss, x, y, N_2, niter=10)
m_fit_3 = fit_lm(m0, gauss, x, y, N_3, niter=10)

on iteration  0  chisq is  0.014252744765107407  with step  [-1.1639515  9.0755122]  and lamda  0
on iteration  1  chisq is  2.0094241849213195e-07  with step  [-0.1796332   0.71192439]  and lamda  0
Converged after  2  iterations of LM
on iteration  0  chisq is  0.016183738401747052  with step  [-1.16396544  9.07556078]  and lamda  0
on iteration  1  chisq is  2.2810769124813567e-07  with step  [-0.17961954  0.71187631]  and lamda  0
Converged after  2  iterations of LM
on iteration  0  chisq is  0.03033129510720546  with step  [-1.16404094  9.07582369]  and lamda  0
on iteration  1  chisq is  4.2691897510919017e-07  with step  [-0.17954562  0.71161604]  and lamda  0
Converged after  2  iterations of LM


In [7]:
y_1, derivs_1 = gauss(m_fit_1, x)
y_2, derivs_2 = gauss(m_fit_1, x) 
y_3, derivs_3 = gauss(m_fit_1, x) 

In [8]:
print(m_fit_1)
print(m_fit_2)
print(m_fit_3)

[500.00000003  49.99999997]
[500.00000003  49.99999997]
[500.00000003  49.99999997]


In [12]:
error_1 = y_1 - y
error_2 = y_2 - y
error_3 = y_3 - y

In [13]:
np.mean(error_1)

-6.535016426205718e-11